In [1]:
import pandas as pd
import ujson
import json
pd.options.display.max_colwidth = 5000
import matplotlib

In [2]:
subreddit_list = ['canada', 'liberal', 'conservative','politics',\
"twoxchromosomes","showerthoughts","tifu"]
political = ['liberal', 'conservative','politics']
nonpolitical = ["twoxchromosomes","showerthoughts","tifu"]
# Read the feather files
data = pd.DataFrame()
for sub in subreddit_list:
    rf = pd.read_feather('../data/feather_files/RS_2019_'+sub+'_df.feather')
    if sub in political:
        rf['class'] = 'political'
    elif sub in nonpolitical:
        rf['class'] = 'nonpolitical'
    elif sub == 'canada':
        rf['class'] = 'test'
    else:
        rf['class'] = 'Unkown'
    data = pd.concat((data,rf))

# Removing posts with less than 10 characters in the body.
data = data[(data.selftext.astype(str).str.len()>10)].reset_index()
print(data.groupby('class').describe(percentiles=[.5]))

# combining titles with the post body. 
data['text'] = data.title + " " + data.selftext
# deleting the previous two columns.
data.drop(columns=["title","selftext"],inplace=True)

                index                                                       \
                count          mean           std   min      50%       max   
class                                                                        
nonpolitical  14804.0   7282.288503   8386.425392   0.0   5001.0   41450.0   
political       885.0  43950.302825  47488.106523  40.0  16476.0  139049.0   
test           2147.0   7482.789474   4476.914083   1.0   7538.0   17225.0   

             num_comments                                                  \
                    count         mean          std   min    50%      max   
class                                                                       
nonpolitical      14804.0   143.823021   457.592274  15.0   26.0  16639.0   
political           885.0  3414.674576  7093.978640  15.0  177.0  52183.0   
test               2147.0    71.351653   204.972232  15.0   31.0   6441.0   

                score                                               

In [3]:
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import random
import swifter

In [4]:
def clean_text(text):
    """
    Input : text
    output : cleaned text
    process:
        1.Remove non-alphabitical words
        2.remove words of less than 3 characters
        3.Remove stopwords
        4.Transform words to lower characters
        4.lemmatize the text - First verbs then nouns
        * Steps are performed in that order.
    """
    tokens = word_tokenize(text)
    lemma = WordNetLemmatizer()
    def clean(word):
        if not word.isalpha() or len(word) < 3:
            return False
        if word.lower() in stopwords.words("english"):
            return False
        return True
    tokens = " ".join(str(x) for x in \
        [lemma.lemmatize(\
            lemma.lemmatize(word.lower(),pos="v")\
            ,pos="n") for word in tokens if clean(word)]\
        )
    return tokens
#--------------------------------------------------
data.text = data.text.swifter.allow_dask_on_strings(enable=True).apply(clean_text)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

In [5]:
data.reset_index().drop(columns=['index'])\
    .to_feather('../data/feather_files/data2019clean.feather')

In [6]:
data

,index,author,created_utc,num_comments,score,domain,locked,subreddit,class,text
0,1,villehockey13,2019-01-01,74,0,self.canada,False,canada,test,argument pipeline hard time find info pipeline hop get clarification hear people say thing like train pollute alberta carry canada economy year province owe ruin environment statement prove otherwise convince someone pipeline good environment
1,4,AutoModerator,2019-01-01,21,23,self.canada,True,canada,test,monthly photography contest reddit gold one suggestion want implement census photography competition contest run every month win image post reddit search bar next month contest end credit give sidebar also keep image future use undecided time whatever reason want retain copy image delete request understand incredibly rare pepes type photography look rule submission must take photograph must permission use photograph submit photograph creation leave original photograph untouched possible add text overlay image etc nothing nsfw allow good luck submission
2,8,xiic,2019-01-01,222,65,self.canada,False,canada,test,illegal specify gender job post get employment equity act http certain class receive special status correct condition disadvantage employment experience woman aboriginal people person disability member visible minority still illegal canadian human right act http discriminate someone base sex wonder first time ever see job post female requirement http
3,10,iphonexmas,2019-01-01,163,2350,self.canada,False,canada,test,grateful live beautiful country happy new year fellow canadian happy new year everyone wish family beautiful full smile positive vibe
4,13,7Trickster,2019-01-01,47,0,self.canada,False,canada,test,canadian need help make decision dear canadian amp firstly happy new year want start give context bear live france since forever work expatriate insurance company age since december time think move elsewhere paris life feel empty bore like go forward live good atmosphere simple man simply wish live peaceful place make family raise child better live condition better life part urge move due fact lose lot friend year various reason somehow want fresh new start see something new simply put life bore enjoy communicate english try look option first candidate due short distance despite felt like would somehow feel like paris worse decide live australia another candidate far family member live portugal spain france austria usa cabo verde compare pro con realize france decent perk better balance work life expensive health care good doctor good food gmos one issue eye direction head identify many european merkel silly decision leadership elite lead disastrous result country germany sweden even france goal get economic migrant big corporation pay le would person country doubt shady objective behind set eye canada never read see lot contradiction online really initial idea come ask guy directly think country overall pact france quebec save could allow specialize study accès etude québec http http roughly one year get chance start new life tempt point question need answer opinion know take decision health care system know suppose free read lot information queue surgery example thing capable go month year beyond bite scary understand private clinic sure many would get rid issue cost job know come cheap north america general tell amp people read lot conflict opinion many say canadian general welcome first everyone others say canadian good heart nice people another group say canadian keep hard make friend new country result kinda difficult guess right course every population positive negative individual want understand people general canada work everyday life amp work condition admit extensive research understand get really better balance least report french people live canada average wage people behave general office etc want know guy could give opinion amp buy apartment save money want buy apartment bedroom barely idea price canada seem quite high saw interest offer apartment look amaze unli